# Fetch Data

Fetches all Anilist data needed for practicing data visualization and machine learning.

In [1]:
import csv
import json
import os
import traceback
from datetime import datetime
from anilist_fetch import fetch as anilist

In [2]:
# config

data_dir = os.path.abspath('data')
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

today = datetime.today().strftime('%Y%m%d')

## Fetch User Data

In [2]:
ANILIST_USER_ID = 247578
user = anilist.get_user(ANILIST_USER_ID)
print(user['user'])

# write user anime entries to CSV
with open(os.path.join(data_dir, f"user-{today}-raw.csv"), 'w+', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['media_id', 'status', 'score', 'progress', 'completedAt'])

    for media_list in user['lists']:
        for entry in media_list['entries']:
            data = [entry['media']['id'], entry['status'], entry['score'], entry['progress'], None]
            completed_at = entry['completedAt']

            if completed_at['year'] and completed_at['month'] and completed_at['day']:
                data[4] = f"{completed_at['month']:02d}-{completed_at['day']:02d}-{completed_at['year']}"

            writer.writerow(data)

{'name': 'barrettotte', 'createdAt': 1552085204, 'statistics': {'anime': {'count': 605, 'meanScore': 68.92, 'standardDeviation': 15.9, 'minutesWatched': 198795, 'episodesWatched': 8138}}}


## Fetch All Anime Entries

Takes around 10-15 minutes to fetch everything while not hitting Anilist's API rate limiter.

In [3]:
anime_count = anilist.download_anime_range(os.path.join(data_dir, f"anime-{today}-raw.csv"))
print(f"\n{anime_count} entries downloaded")

16551 entries downloaded
